In [2]:
# solid state nmr Hamiltonian
import numpy as np
import matplotlib.pyplot as plt

# Define constants
gamma = 42.577  # Gyromagnetic ratio for 1H in MHz/T
magic_angle = np.arccos(1 / np.sqrt(3))  # Magic angle in radians
spinning_frequency = 10.0  # Spinning frequency in kHz
num_orientations = 1000  # Number of powder orientations

# Principal axis frame (CSA principal values in ppm)
sigma_11 = 10.0
sigma_22 = 50.0
sigma_33 = 70.0
sigma_iso = (sigma_11 + sigma_22 + sigma_33) / 3

# Generate random powder orientations (theta, phi)
np.random.seed(42)  # For reproducibility
theta = np.arccos(np.random.uniform(-1, 1, num_orientations))  # [0, π]
phi = np.random.uniform(0, 2 * np.pi, num_orientations)  # [0, 2π]

# Transition from Principal Axis Frame to Molecular Frame
def csa_tensor_principal(sigma_11, sigma_22, sigma_33):
    """Construct the CSA tensor in the principal axis frame."""
    return np.diag([sigma_11, sigma_22, sigma_33])

def rotation_matrix(theta, phi):
    """Return a rotation matrix for Euler angles theta, phi."""
    return np.array([
        [np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)],
        [np.cos(theta) * np.cos(phi), np.cos(theta) * np.sin(phi), -np.sin(theta)],
        [-np.sin(phi), np.cos(phi), 0]
    ])

# Compute MAS frequencies
frequencies = []
for t, p in zip(theta, phi):
    # Transform from Principal Axis Frame to Molecular Frame
    R_pm = rotation_matrix(t, p)
    sigma_mol = R_pm @ csa_tensor_principal(sigma_11, sigma_22, sigma_33) @ R_pm.T
    
    # Transform to Rotor Frame (spinning about the magic angle)
    R_mr = rotation_matrix(magic_angle, 0)
    sigma_rotor = R_mr @ sigma_mol @ R_mr.T
    
    # Transform to Laboratory Frame
    sigma_lab = sigma_rotor  # For simplicity, assume lab frame coincides with rotor frame
    
    # Compute anisotropic shift under MAS
    delta_sigma = sigma_lab[2, 2] - sigma_iso
    frequency = delta_sigma * spinning_frequency
    frequencies.append(frequency)

# Simulate spectrum
bins = 1000
hist, edges = np.histogram(frequencies, bins=bins, range=(-200, 200), density=True)
spectrum = np.convolve(hist, np.exp(-np.linspace(-2, 2, 10)**2), mode="same")

# Plot the spectrum
plt.figure(figsize=(8, 4))
plt.plot(edges[:-1], spectrum, label="MAS Spectrum", color="blue")
plt.xlabel("Frequency (kHz)")
plt.ylabel("Intensity (a.u.)")
plt.title("Simulated MAS NMR Spectrum")
plt.legend()
plt.grid()
plt.show()
